In [1]:
import visual_behavior_glm.src.GLM_params as glm_params
import visual_behavior_glm.src.GLM_analysis_tools as gat
import visual_behavior_glm.src.GLM_visualization_tools as gvt
from visual_behavior_glm.src.glm import GLM
import matplotlib.pyplot as plt
import visual_behavior.data_access.loading as loading
import visual_behavior.database as db
import plotly.express as px

import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import os

import seaborn as sns

/home/dougo/Code/AllenSDK/allensdk/brain_observatory/behavior/behavior_ophys_api/behavior_ophys_nwb_api.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [14]:
glm_version = '6_L2_optimize_by_session'

In [2]:
experiment_table = loading.get_filtered_ophys_experiment_table()

In [11]:
ms_session = experiment_table.query('equipment_name == "MESO.1"').sample(random_state= 0).reset_index()
ms_session

,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,has_events,session_number
0,964615764,963749922,964008613,1018027744,VisualBehaviorMultiscope4areasx2d,holding,passed,20191009_472271_Ophys6_2,OPHYS_6_images_H,MESO.1,2019-10-09 10:44:31.238545,910867827,900263316,M,154.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",289,VISam,NaN,900263316,Slc17a7-IRES2-Cre,NaN,NaN,0,False,Slc17a7_VISam_289,True,6


In [12]:
scientifica_session = experiment_table.query('equipment_name == "CAM2P.3"').sample(random_state= 0).reset_index()
scientifica_session

,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,has_events,session_number
0,965928394,965319434,965505185,969421516,VisualBehaviorTask1B,container_qc,passed,20191011_467954_6ImagesA,OPHYS_6_images_A,CAM2P.3,2019-10-11 15:15:53.000000,897688827,887127198,M,178.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],175,VISp,NaN,887127198,Vip-IRES-Cre,NaN,NaN,0,True,Vip_VISp_175,True,6


In [37]:
W = {}

In [40]:
%%time
W['mesoscope'] = gat.get_weights_matrix_from_mongo(int(ms_session['ophys_experiment_id'][0]), glm_version)

CPU times: user 21 ms, sys: 1.81 ms, total: 22.8 ms
Wall time: 90.8 ms


In [36]:
%%time
W['scientifica'] = gat.get_weights_matrix_from_mongo(int(scientifica_session['ophys_experiment_id'][0]), glm_version)

CPU times: user 20.5 ms, sys: 0 ns, total: 20.5 ms
Wall time: 54.1 ms


In [20]:

W['mesoscope'] = Ro(int(ms_session['ophys_experiment_id'][0]), glm_version)
W['scientifica'] = gat.get_weights_matrix_from_mongo(int(scientifica_session['ophys_experiment_id'][0]), glm_version)

In [41]:
W['mesoscope']

<xarray.DataArray (weights: 629, cell_specimen_id: 18)>
array([[-0.0029417 ,  0.00289943,  0.00071985, ..., -0.00032836,
         0.01075657,  0.0017955 ],
       [ 0.06092284, -0.00201001, -0.00395869, ..., -0.0017579 ,
         0.00674091,  0.00128063],
       [ 0.01012048, -0.00050152,  0.00144199, ...,  0.00218003,
         0.00300574, -0.00036021],
       ...,
       [-0.02483475, -0.00854851, -0.00148955, ...,  0.00318787,
         0.01113922,  0.00781823],
       [-0.01429384, -0.01858457, -0.00046724, ..., -0.00324818,
         0.01639156,  0.00305253],
       [-0.01173425, -0.01593511,  0.00325082, ...,  0.0025355 ,
         0.01272949,  0.0040504 ]])
Coordinates:
  * weights           (weights) <U20 'intercept_0' 'time_0' ... 'image7_8'
  * cell_specimen_id  (cell_specimen_id) int64 1016939431 ... 1016940031

In [23]:
W['mesoscope'].values.shape

(629, 18)

In [24]:
W['scientifica'].values.shape

(1841, 28)

In [31]:
W['mesoscope']['weights'].values

array(['intercept_0', 'time_0', 'pre_licks_-54', 'pre_licks_-53',
       'pre_licks_-52', 'pre_licks_-51', 'pre_licks_-50', 'pre_licks_-49',
       'pre_licks_-48', 'pre_licks_-47', 'pre_licks_-46', 'pre_licks_-45',
       'pre_licks_-44', 'pre_licks_-43', 'pre_licks_-42', 'pre_licks_-41',
       'pre_licks_-40', 'pre_licks_-39', 'pre_licks_-38', 'pre_licks_-37',
       'pre_licks_-36', 'pre_licks_-35', 'pre_licks_-34', 'pre_licks_-33',
       'pre_licks_-32', 'pre_licks_-31', 'pre_licks_-30', 'pre_licks_-29',
       'pre_licks_-28', 'pre_licks_-27', 'pre_licks_-26', 'pre_licks_-25',
       'pre_licks_-24', 'pre_licks_-23', 'pre_licks_-22', 'pre_licks_-21',
       'pre_licks_-20', 'pre_licks_-19', 'pre_licks_-18', 'pre_licks_-17',
       'pre_licks_-16', 'pre_licks_-15', 'pre_licks_-14', 'pre_licks_-13',
       'pre_licks_-12', 'pre_licks_-11', 'pre_licks_-10', 'pre_licks_-9',
       'pre_licks_-8', 'pre_licks_-7', 'pre_licks_-6', 'pre_licks_-5',
       'pre_licks_-4', 'pre_licks_-3', 